# Clasificación supervisada

In [12]:
## Importación de librerías

import spacy
import pandas as pd

nlp_español = spacy.load('es_core_news_lg')  

In [13]:
## Lectura de datos

datos = pd.read_csv('hotel.csv')
print(datos.head())
print(datos.shape)
print(datos.columns)

                                                text  label
0  Es un gran hotel; el mejor de Asunción. Buenas...      3
1  hola. no suelo criticar jamas lo que paso pero...      3
2  Escogi meses antes de mi boda una habitacion p...      3
3  Voy a se Lo mas equitativo posible; porque soy...      3
4  Esta es una experiencia de septiembre de 2016;...      3
(200, 2)
Index(['text', 'label'], dtype='object')


In [14]:
### Preprocesamiento y normalización

In [15]:
# Vamos a separar los documentos y sus categorías. 
# docs y categs son series de Pandas
# Hay que separar las categorías de los documentos para usar 
# estos últimos y obtener la matriz BOW o Tf-idf

docs = datos.iloc[:,0] # extract column with review
categs = datos.iloc[:,-1] # extract column with sentiment

In [16]:
print("Datos es tipo: ", type(datos))
print("Docs es tipo: ", type(docs))
print("Categs es tipo: ", type(categs))

Datos es tipo:  <class 'pandas.core.frame.DataFrame'>
Docs es tipo:  <class 'pandas.core.series.Series'>
Categs es tipo:  <class 'pandas.core.series.Series'>


## Obtención de las matrices BOW y Tf-idf

In [17]:
# tokenizamos los documentos y convertimos en matriz BOW

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
### vectorizer = CountVectorizer(max_features=20)
docs_vec = vectorizer.fit_transform(docs)
docs_vec

<200x2426 sparse matrix of type '<class 'numpy.int64'>'
	with 8710 stored elements in Compressed Sparse Row format>

In [18]:
docs_vec_densa = docs_vec.todense() # convierte la matriz en densa
docs_vec_densa

matrix([[0, 0, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [19]:
# Construimos la matriz formato Tf-idf

from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer() 
docs_tfidf = tfidf.fit_transform(docs_vec)
docs_tfidf_densa = docs_tfidf.todense()
docs_tfidf_densa

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.11515974,
         0.        ],
        [0.04403155, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

## Preparación de los subconjuntos de entrenamiento y test

In [20]:
# División mediante train_test_split. Test de 25%

from sklearn.model_selection import train_test_split
docs_train, docs_test, categs_train, categs_test = train_test_split(docs_tfidf, categs, test_size = 0.25, 
                                                                    random_state = 50)

In [21]:
print(docs_train)

  (0, 2411)	0.0547537283984435
  (0, 2407)	0.032281077018586145
  (0, 2369)	0.05079649092264503
  (0, 2365)	0.0547537283984435
  (0, 2363)	0.044031548775208415
  (0, 2362)	0.04798878625100688
  (0, 2359)	0.03648540993257896
  (0, 2347)	0.109507456796887
  (0, 2344)	0.03811581578278578
  (0, 2319)	0.04007431129940993
  (0, 2318)	0.24562147183157004
  (0, 2313)	0.040389711979148195
  (0, 2312)	0.06476233396107638
  (0, 2310)	0.0547537283984435
  (0, 2309)	0.05079649092264503
  (0, 2288)	0.045810961313459395
  (0, 2281)	0.0547537283984435
  (0, 2262)	0.04122384410357026
  (0, 2255)	0.0547537283984435
  (0, 2248)	0.0547537283984435
  (0, 2245)	0.020194855989574097
  (0, 2238)	0.0547537283984435
  (0, 2237)	0.04798878625100688
  (0, 2231)	0.0547537283984435
  (0, 2230)	0.0547537283984435
  :	:
  (149, 1305)	0.38101705377200434
  (149, 1293)	0.0889977050590928
  (149, 1175)	0.2928784826262668
  (149, 1151)	0.1464392413131334
  (149, 1113)	0.20534985683535834
  (149, 1108)	0.16513710575890536

## Support Vector Machine

In [23]:
# Entrenamiento del clasificador NB

from sklearn.svm import SVC
svm = SVC(kernel='linear')
# svm = SVC(kernel='poly')
# svm = SVC(kernel='rbf')
# svm = SVC(kernel='sigmoid')


svm.fit(docs_train, categs_train)

SVC(kernel='linear')

## Evaluación del modelo

In [24]:
# Predicción del set de test

categs_pred = svm.predict(docs_test)

In [25]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(categs_test, categs_pred)
cm

array([[18,  7],
       [11, 14]])

In [26]:
acc_train = svm.score(docs_train, categs_train)
acc_test = svm.score(docs_test, categs_test)

print("Accuracy entrenamiento: ", acc_train)
print("Accuracy PRUEBA: ", acc_test)
print("Fiabilidad: ", acc_test / acc_train)  

Accuracy entrenamiento:  0.9933333333333333
Accuracy PRUEBA:  0.64
Fiabilidad:  0.6442953020134229
